In [5]:
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from torchvision import transforms
import os


In [9]:
m_path = '/home/tungnguyendinh/.fastai/data/oxford-iiit-pet'
print(os.listdir(m_path))

['images', 'annotations']


In [10]:
df = pd.read_csv(f"{m_path}/annotations/trainval.txt", sep = " ", header=None)
df.columns = ["file", "class", "type", "breed"]
df.head()

,file,class,type,breed
0,Abyssinian_100,1,1,1
1,Abyssinian_101,1,1,1
2,Abyssinian_102,1,1,1
3,Abyssinian_103,1,1,1
4,Abyssinian_104,1,1,1


In [ ]:
class OxfordPetDataset(Dataset):
    def __init__(self, img_dir, annotations_file, transform = None, target_transform = None)

In [ ]:
class PascalVOCDataModule(pl.LightningDataModule):
    def __init__(self, batch_size, data_dir):
        super().__init__()
        self.batch_size = batch_size
        self.data_dir = data_dir
        self.mean = [0.457342265910642, 0.4387686270106377, 0.4073427106250871]
        self.std = [0.26753769276329037, 0.2638145880487105, 0.2776826934044154]  
        self.train_transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((300, 300)),
                transforms.RandomChoice([
                                        transforms.ColorJitter(brightness=(0.9, 1.1)),
                                        transforms.RandomGrayscale(p = 0.25)
                                        ]),
                transforms.RandomHorizontalFlip(p = 0.25),
                transforms.RandomRotation(25),
                transforms.ToTensor(), 
                transforms.Normalize(mean = self.mean, std = self.std),
                ])
        self.eval_transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize((300, 300)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean = self.mean, std = self.std)
                   ])
        
    def setup(self):
        stage = "train"
        self.voc_train = PascalDataset(f"{self.data_dir}/{stage}", f"{self.data_dir}/{stage}.csv", transform=self.train_transform, is_train= True)
        self.voc_val = PascalDataset(f"{self.data_dir}/{stage}", f"{self.data_dir}/{stage}.csv", transform=self.eval_transform, is_valid = True) 
        stage = "test"                       
        self.voc_test = PascalDataset(f"{self.data_dir}/{stage}", f"{self.data_dir}/{stage}.csv", transform=self.eval_transform)
            
    def train_dataloader(self):
        self.setup()
        return DataLoader(self.voc_train, self.batch_size)
    
    def val_dataloader(self):
        return DataLoader(self.voc_val, self.batch_size)
    
    def test_dataloader(self):
        return DataLoader(self.voc_test, self.batch_size)